# Tennis Lab Core Components

This notebook demonstrates the core components of the `tennis_lab` framework for modeling tennis matches. The framework provides a hierarchical structure that mirrors real tennis scoring:

- **MatchFormat** - Defines the rules of a match (best of 3/5, tiebreak rules, no-ad scoring, etc.)
- **Game** / **GameScore** - A single game within a set
- **Tiebreak** / **TiebreakScore** - A tiebreak game (regular or super)
- **Set** / **SetScore** - A set within a match
- **Match** / **MatchScore** - A complete match

Each component can be initialized at any valid starting score, allowing you to model scenarios mid-match.

In [ ]:
import os, sys

# Add path to the 'tennis_lab' package if not in PYTHONPATH already 
PROJECT_ROOT = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..'))
SRC_DIR = os.path.join(PROJECT_ROOT, 'src')
if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

from tennis_lab.core.game           import Game
from tennis_lab.core.game_score     import GameScore
from tennis_lab.core.match          import Match
from tennis_lab.core.match_score    import MatchScore
from tennis_lab.core.match_format   import MatchFormat, SetEnding
from tennis_lab.core.set            import Set
from tennis_lab.core.set_score      import SetScore
from tennis_lab.core.tiebreak       import Tiebreak
from tennis_lab.core.tiebreak_score import TiebreakScore

## 1. MatchFormat - Defining the Rules

Before creating any match components, you must define the format/rules using `MatchFormat`. This controls:
- Number of sets (best of 3 or 5)
- How the match ends (a final set or a super-tiebreak)
- How sets end when tied (tiebreak or advantage)
- How many games in a set (6 is standard, 8 for pro-sets, 4 in some junior turnaments)
- Whether games use the "no-ad" rule (sudden death at deuce)
- etc.

In [ ]:
print("\n" + "="*30 + "\n")

# US Open ATP format
USOpen_format = MatchFormat(
    bestOfSets     = 5, 
    matchTiebreak  = False,
    setEnding      = SetEnding.TIEBREAK,
    finalSetEnding = SetEnding.TIEBREAK
)
print("US Open Men - ", end='')
print(USOpen_format)

print("\n" + "="*30 + "\n")

# Wimbledon ATP format
Wimbledon_format = MatchFormat(
    bestOfSets     = 5, 
    matchTiebreak  = False,
    setEnding      = SetEnding.TIEBREAK,
    finalSetEnding = SetEnding.SUPERTIEBREAK
)
print("Wimbledon Men - ", end='')
print(Wimbledon_format)

print("\n" + "="*30 + "\n")

# Doubles WTA format
WTA_doubles_format = MatchFormat(
    bestOfSets    = 3, 
    matchTiebreak = True,
    setEnding     = SetEnding.TIEBREAK,
)
print("WTA doubles - ", end='')
print(WTA_doubles_format)

print("\n" + "="*30 + "\n")

# Junior singles tournaments

junior_noad_format = MatchFormat(
    bestOfSets    = 3, 
    matchTiebreak = True,
    setEnding     = SetEnding.TIEBREAK,
    noAdRule      = True
)
print("Junior no-ad - ", end='')
print(junior_noad_format)

print("\n" + "="*30 + "\n")

junior_proset = MatchFormat(
    bestOfSets    = 1, 
    setLength     = 8,
    matchTiebreak = False,
    setEnding     = SetEnding.TIEBREAK,
    noAdRule      = True
)
print("Junior pro-set - ", end='')
print(junior_proset)

print("\n" + "="*30 + "\n")

## 2. Game - The Basic Unit

A `Game` represents a single game within a set. Key features:
- Tracks the server and current score
- Handles deuce/advantage logic (or no-ad rule)
- Can start from any valid score (not just 0-0)
- Records point-by-point history

In [ ]:
# Create a new game - Player 1 serves, starting at 0-0
# Note: matchFormat is optional, defaults to standard scoring
game = Game(playerServing=1)
print(f"Initial state: {game}")
print(f"Server: Player {game.server}")
print(f"Is over? {game.isOver}")

# Play some points
game.recordPoint(pointWinner=1)  # Player1 wins
print(f"\nAfter P1 wins point: {game}")

game.recordPoint(pointWinner=2)  # Player2 wins
print(f"After P2 wins point: {game}")

# Continue playing until the game is over (Player1 wins remaining points)
print("\nContinuing until game is over:")
while not game.isOver:
    game.recordPoint(1)
    print(f"  {game}")
    
print(f"\nWinner: Player {game.winner}")
print(f"Points played: {len(game.pointHistory)}")

In [ ]:
# Game with deuce scenario
game_deuce = Game(playerServing=2)

# Get to deuce (40-40)
game_deuce.recordPoints([1, 1, 1, 2, 2, 2])  
print(f"Score is deuce? - {game_deuce.score.isDeuce}")
print(f"{game_deuce}")

# Advantage Player2, then back to deuce
game_deuce.recordPoint(2) 
print(f"\nPlayer with advantage: {game_deuce.score.playerWithAdvantage}")
print(f"{game_deuce}")
game_deuce.recordPoint(1) 
print(f"Score is deuce? - {game_deuce.score.isDeuce}")
print(f"{game_deuce}")

# Finally win by Player1
game_deuce.recordPoints([1, 1])  # Player1 wins the next  points
print(f"\nFinal: {game_deuce}")
print(f"\nScore history:\n{game_deuce.scoreHistory}")

In [ ]:
# No-ad scoring: sudden death at deuce
noad_format = MatchFormat(noAdRule=True)
game_noad   = Game(playerServing=1, matchFormat=noad_format)

# Get to deuce
game_noad.recordPoints([1, 1, 1, 2, 2, 2])
print(f"At deuce (no-ad):  {game_noad}")

# Next point wins!
game_noad.recordPoint(2)
print(f"One point decides: {game_noad}")
print(f"Winner: Player {game_noad.winner}")

In [ ]:
# Starting from a non-zero score (e.g., analyzing a specific scenario)
init_score = GameScore(pointsP1=2, pointsP2=1)  # 30-15
game_mid   = Game(playerServing=1, initScore=init_score)
print(f"Starting mid-game: {game_mid}\n")
print(f"Score as points from P1 perspective: {game_mid.score.asPoints(pov=1)}")
print(f"Score as points from P2 perspective: {game_mid.score.asPoints(pov=2)}")
print(f"Score traditional: {game_mid.score.asTraditional(pov=1)}")

# Two more points for P1 wins
game_mid.recordPoints([1, 1])
print(f"\nFinal: {game_mid}")
print(f"Point history (only points played, not init): {game_mid.pointHistory}")

## 3. Tiebreak

A `Tiebreak` is a special game format used to decide a set when tied at 6-6. Key differences from regular games:
- First to 7 points (or 10 for super-tiebreak), win by 2
- Server alternates every 2 points (after the first point)
- Points are counted numerically (0, 1, 2, ...) not traditionally

In [ ]:
# Regular tiebreak (first to 7)
tb = Tiebreak(playerServing=1, isSuper=False)
print(f"Initial: {tb}")
print(f"First server: Player {tb.servesNext}")

# Play a few points, showing server rotation
for i in range(6):
    server_before = tb.servesNext
    tb.recordPoint(pointWinner=(1 if i % 2 == 0 else 2))
    print(f"Point {i+1}: P{server_before} served → Score {tb.score.asPoints(1)} → Next server: P{tb.servesNext}")

print(f"\nCurrent state: {tb}")

In [ ]:
# Super-tiebreak (first to 10) - often used as match tiebreak in doubles
super_tb = Tiebreak(playerServing=2, isSuper=True)
print(f"Super-tiebreak: {super_tb}")

# Simulate a close super-tiebreak: play to 10-8 (Player1 wins)
points = [1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,1,1,1] 
super_tb.recordPoints(points)

print(f"Final: {super_tb}")
print(f"Winner: Player {super_tb.winner}")
print(f"\nScore history:\n{super_tb.scoreHistory}")

## 4. Set

A `Set` manages multiple games and handles:
- Server rotation between games
- Tiebreak triggering at 6-6 (configurable)
- Tracking games won by each player
- Game-by-game score history

In [ ]:
import random
random.seed(42)  # For reproducibility

# Create a new set - Player 1 serves first
# Note: 'set' is a Python reserved word, so we use 'tennis_set'
tennis_set = Set(playerServing=1, isFinalSet=False, matchFormat=MatchFormat())
print(f"Initial: {tennis_set}")

# Simulate point-by-point playing until the set is over
while not tennis_set.isOver:

    # Server wins point with 60% probability
    server = tennis_set.servesNext
    point_winner = server if random.random() < 0.60 else (3 - server)
    tennis_set.recordPoint(point_winner)

print(f"Final:   {tennis_set}")
print(f"Winner:  Player {tennis_set.winner}")
print(f"Points won: Player1={tennis_set.totalPoints[0]}, Player2={tennis_set.totalPoints[1]}")

In [ ]:
# View the detailed score history
print("Set Score History:")
print(tennis_set.scoreHistory())

In [ ]:
# Start a set mid-way (e.g., at 5-4 with current game at 30-15)
matchFormat = MatchFormat()
isFinalSet  = False

mid_set_score = SetScore(
    gamesP1=5, gamesP2=4, 
    isFinalSet=isFinalSet, 
    matchFormat=matchFormat,
    gameScore=GameScore(2, 1, matchFormat)  # 30-15 in current game
)
mid_set = Set(playerServing=2, isFinalSet=isFinalSet, initScore=mid_set_score)
print(f"Starting mid-set: {mid_set}")

# P2 holds serve to make it 5-5
mid_set.recordPoints([2, 2, 2])  # Player2 wins the next 3 points
print(f"After P2 holds:   {mid_set}")

## 5. Match

A `Match` orchestrates multiple sets and provides:
- Complete point-by-point tracking across all sets
- Automatic set transitions
- Full match score history
- Support for various match formats

In [ ]:
# Create a best-of-3 match
match_format = MatchFormat(bestOfSets=3)
match = Match(playerServing=1, matchFormat=match_format)
print(f"Initial: {match}")

# Simulate the match with slightly better server
random.seed(123)
while not match.isOver:
    server = match.servesNext
    # Server wins 62% of points
    point_winner = server if random.random() < 0.62 else (3 - server)
    match.recordPoint(point_winner)

print(f"\nFinal: {match}")
print(f"Match winner: Player {match.winner}")
print(f"Total points played: {sum(match.totalPoints)}")

In [ ]:
# View detailed match history
print("Match Score History:")
print(match.scoreHistory())

In [ ]:
# Start a match at a specific score (e.g., analyzing a critical moment)
# 5th set, 6-5, 40-30, Player1 serving
MF = MatchFormat(bestOfSets=5)
critical_moment = MatchScore(
    setsP1=2, setsP2=2,
    setScore=SetScore(
        gamesP1=6, gamesP2=5,
        isFinalSet=True,
        gameScore=GameScore(3, 2, MF)  # 40-30
    )
)
clutch_match = Match(playerServing=1, initScore=critical_moment)
print(f"Critical moment: {clutch_match}")
print("P1 is serving for the match at 40-30...")

# P1 wins the point and the match!
clutch_match.recordPoint(1)
print(f"\nResult: {clutch_match}")

## 6. Replaying a Match from Point History

One powerful feature is the ability to replay an entire match from its point history. This is useful for:
- Analyzing recorded matches
- Monte Carlo simulations
- What-if scenarios

In [ ]:
# Get the point history from our simulated match
original_points = match.pointHistory
print(f"Original match had {len(original_points)} points")
print(f"First 20 points: {original_points[:20]}...")

# Replay the exact same match
replayed = Match(playerServing=1, matchFormat=match.matchFormat)
replayed.recordPoints(original_points)

print(f"\nOriginal result: {match}")
print(f"Replayed result: {replayed}")
print(f"Results match: {match.winner == replayed.winner}")